## Ingest pitstops.json file

In [0]:
# here the pitstops.json is a multiline json and and not a single line json or nested json
display(dbutils.fs.ls('/mnt/formulaf1adls/raw/'))

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType

pit_stops_schema = StructType( fields= [StructField('raceId', IntegerType(), False),
                                        StructField('driverId', IntegerType(), True),
                                        StructField('stop', StringType(), True),
                                        StructField('lap', IntegerType(), True),
                                        StructField('time', StringType(), True),
                                        StructField('duration', StringType(), True),
                                        StructField('milliseconds', IntegerType(),True)
                                        ])

pit_stops_df = spark.read \
.schema(pit_stops_schema) \
.option('multiLine', True) \
.json('dbfs:/mnt/formulaf1adls/raw/pit_stops.json')

pit_stops_df.limit(5).display()

In [0]:
from pyspark.sql.functions import col, current_timestamp

pit_stops_renamed_df = pit_stops_df.withColumnRenamed('raceId', 'race_id') \
                                   .withColumnRenamed('driverId', 'driver_id') \
                                   .withColumn('ingestion_date', current_timestamp())

pit_stops_renamed_df.limit(10).display()

In [0]:
pit_stops_renamed_df.write.mode('overwrite').parquet('dbfs:/mnt/formulaf1adls/processed/pit_stops')

In [0]:
display(spark.read.parquet('dbfs:/mnt/formulaf1adls/processed/pit_stops'))